# BANK MARKETING PROJECT

The following project focus on the analysis of a dataset 'Bank Marketing' which contains data about customers and aims to get useful insights from the data and predict if a new customer will accept a deposit offer or not.<br>

The project is structured as follows:

- Data Cleaning + Feature Engineering
- Exploratory Data Analysis
- Data preparation for ML algorithms (encoding)
- XGBOOST training and Hyperparameter optimization
- Results Summary

# Campaign Success Prediction results dashboard:

**NOTE: The dataset contains lots of missing, unknown or 'other' values for categorical features, which is bad for prediction. It is recommended to the company to collect better the data and have less uncertain values.**

- XGBoost achives good results in terms of precision, recall and accuracy, with values around 70%.
- The most important features to determine if a new customer will accept the deposit is poutcome (>30%):<br>
    - **This means that the deposit satisfied the customers who accepted it previously, however, currently, the company is probably aiming at the 'wrong' customers** 
 <br>
- Moreover, accoring to XGBoost, other important features to predict the success of a deposit are:
    - Contact: by looking at the data it looks like the great majority of customers with unknown contact did not accept the deposit: it should be investigated why these contacts are missing.
    - Housing: customers without housing loans seems to accept the deposit

<img src="https://i.imgur.com/8xFCrIJ.png" width="1000px">

# Bank Marketing Analysis results dashboard:

Main results from the data analysis:
- Customers who previously accepted the deposit tends to accept the deposit.
- Customers with job type as 'student' or 'retired' tends to accept the deposit, while blue collars tends to refuse the deposit.
- Customers without a personal loan tends to accept the deposit.
- Customers without a Housing Loan tends to accept the deposit.
- Customers tends to accept more the deposit on march, april, september and october.

<img src="https://i.imgur.com/3s4oCPA.png" width="1000px">

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
  
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score, auc, roc_curve


import xgboost as xgb

import optuna

seed=42

plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlesize=14, titlepad=10)

## Custom functions definition

In [ ]:
def num_plot(df, col, title, symb):
    fig, ax = plt.subplots(2, 1, sharex=True, figsize=(8,5),gridspec_kw={"height_ratios": (.2, .8)})
    ax[0].set_title(title,fontsize=18)
    sns.boxplot(x=col, data=df, ax=ax[0],color = "#1D8EF5")
    ax[0].set(yticks=[])
    sns.histplot(x=col, data=df, ax=ax[1],color = "#33AAFF", edgecolor="#1D1EA2")
    ax[1].set_xlabel(col, fontsize=16)
    plt.axvline(df[col].mean(), color='darkgreen', linestyle='--',linewidth=2.2, label='mean=' + str(np.round(df[col].mean(),1)) + symb)
    plt.axvline(df[col].median(), color='red', linestyle='--',linewidth=2.2, label='median='+ str(np.round(df[col].median(),1)) + symb)
    plt.axvline(df[col].mode()[0], color='purple', linestyle='--',linewidth=2.2, label='mode='+ str(np.round(df[col].mode()[0],1)) + symb)
    plt.legend(bbox_to_anchor=(1, 1.03), ncol=1, fontsize=17, fancybox=True, shadow=True, frameon=True)
    plt.tight_layout()
    plt.show()

In [ ]:
df=pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')

**Dataset information:**
#### bank client data:
1 - age (numeric)<br>
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')<br>
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)<br>
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')<br>
5 - default: has credit in default? (categorical: 'no','yes','unknown')<br>
6 - balance: bank balance <br>
7 - housing: has housing loan? (categorical: 'no','yes','unknown')<br>
8 - loan: has personal loan? (categorical: 'no','yes','unknown')<br>
#### Info related with the last contact of the current campaign:
9 - contact: contact communication type (categorical: 'cellular','telephone')<br>
10 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')<br>
11 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')<br>
12 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.<br>
#### other attributes:
13 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)<br>
14 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric;-1  means client was not previously contacted)<br>
15 - previous: number of contacts performed before this campaign and for this client (numeric)<br>
16 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')<br>

Output variable (desired target):<br>
17 - deposit - has the client subscribed a term deposit? (binary: 'yes','no')<br>



In [ ]:
df.head()

We will first create a copy called 'df' of the original 'df1' dataset where we will perform data cleaning.

In [ ]:
df.columns

In [ ]:
df.info()

There no missing values and there are a lot of categorical features in the data. They will be analyzed first and then encoded before the ML training.

In [ ]:
df.describe()

Some first insights on the data:<br>
- Age:
    - the medium age is 41 years old.
    - the minimum age is 18 years old.
    - the maximum age is 95 years old.
- Balance:
    - The std deviation looks high (3225) compared to the mean (1528).<br>
- Housing and deposit look balanced as their mean value is close to 0.5
- From the column 'previous' we can see that most of the clients have been already contacted, since the value is close to 1 (0.8325)
- Day should be converted to object type

In [ ]:
df['day'] = df['day'].astype(object)

As written in the dataset description, the duration column should be dropped since it includes an information which cannot be known before the call to the client.

In [ ]:
df=df.drop('duration', axis=1)

# Data Cleaning and Feature Engineering

First, we create a copy of the original dataframe where we will perform some data cleaning.

In [ ]:
df_clean = df.copy()

In [ ]:
df_clean.describe()

In [ ]:
sns.heatmap(df.corr(), cmap='RdBu', annot=True, vmin=-1, vmax=1)
plt.title('Correlation Matrix')
plt.show()

From the correlation matrix, the only interesting correlation is the moderate positive correlation (+0.51) between 'previous' and pdays'.

## Age analysis

In [ ]:
num_plot(df_clean, 'age', 'Age Distribution', 'yo')

The distribution of 'Age' is right skewed, with similar value of mean and median and a lower mode.

## Balance Analysis

In [ ]:
num_plot(df_clean, 'balance', 'Balance Distribution', '$')

The distribution is heavily right skewed, with some negative values as well.

## Campaign analysis

In [ ]:
num_plot(df_clean, 'campaign', 'Campaign Distribution', '')

This variable presents a long tail distribtion, as the previous ones.

## pdays Analysis

In [ ]:
num_plot(df_clean, 'pdays', 'Pdays Distribution', 'days')

The great majority of customers in the dataset have pdays=-1 (not previously contacted). We could create a new binary feature to indentify customers who have previosuly been contacted or not.

In [ ]:
df_clean['new_cust'] = df_clean['pdays'] == -1

In [ ]:
df_clean['new_cust'] = df_clean['new_cust'].map({False:0, True:1})

# Exploratory Data Analysis

In [ ]:
df=df_clean

To allow a better visualization, different palette will be chosen for the following plots.

In [ ]:
palette = {'yes':'green', 'no':'red'}
palette = {'yes':'#46C646', 'no':'#FF0000'}

## Job Vs deposit

In [ ]:
plt.figure(figsize=(8, 5.5));
plt.title('Job vs campaign success')
g = sns.countplot(x= 'job', hue = 'deposit', data=df, palette=palette)
plt.xticks(rotation=70)
plt.yticks([])
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We can see that customers with job type as 'student' or 'retired' tends to accept the deposit, while 'blue collars' largely refused the deposit.**

## Marital Status vs campaign success

In [ ]:
plt.figure(figsize=(5, 4.5));
plt.title('Marital Status vs campaign success')
g = sns.countplot(x= 'marital', hue = 'deposit', data=df, palette=palette);
plt.xticks(rotation=70)
plt.yticks([])
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We cannot draw any particular conclusion.**

## Education vs campaign success

In [ ]:
plt.figure(figsize=(5, 4));
plt.title('Education vs campaign success')
g = sns.countplot(x= 'education', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We cannot draw any particular conclusion.**

## Default vs campaign success

In [ ]:
plt.figure(figsize=(4, 4));
plt.title('Default vs campaign success')
g = sns.countplot(x= 'default', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.yticks([])
plt.show()

**We do not have enough data for customers with default=1 to draw a conclusion.**

## Loan vs campaign success

In [ ]:
plt.figure(figsize=(4, 4));
plt.title('Loan vs campaign success')
g = sns.countplot(x= 'loan', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.yticks([])
plt.show()

**It looks like that customers with a personal loan tends to refuse the deposit**

## Contact vs campaign success

In [ ]:
plt.figure(figsize=(4, 5));
plt.title('Contact vs campaign success')
g = sns.countplot(x= 'contact', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We cannot draw any particular conclusion.**

## Housing vs campaign success

In [ ]:
plt.figure(figsize=(4, 5));
plt.title('Housing vs campaign success')
g = sns.countplot(x= 'housing', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**Customers without a Housing Loan tends to accept the deposit.**

## Day vs campaign success

In [ ]:
plt.figure(figsize=(18, 4));
plt.title('Day vs campaign success')
g = sns.countplot(x= 'day', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We cannot draw any particular conclusion.**

## Month vs campaign success

In [ ]:
plt.figure(figsize=(8, 4));
plt.title('Month vs campaign success')
g = sns.countplot(x= 'month', hue = 'deposit', data=df, order=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'], palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**It looks like that customers tends to accept more the deposit on march, april, september and october.**

## Poutcome vs campaign success

In [ ]:
plt.figure(figsize=(5, 4));
plt.title('Poutcome vs campaign success')
g = sns.countplot(x= 'poutcome', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We can see that customers who previously accepted the deposit tends to accept the deposit by a large margin.**

## New_cust VS campaign success

In [ ]:
plt.figure(figsize=(5, 4));
plt.title('New_customer vs campaign success')
g = sns.countplot(x= 'new_cust', hue = 'deposit', data=df, palette=palette);
plt.legend(title='Succes?', ncol=1, fancybox=True, shadow=True)
plt.yticks([])
for container in g.containers:
    g.bar_label(container, padding = 2)
plt.show()

**We can see that customers who had previously been contacted tends to accept the deposit**

Is there a relationship between balance and age?

In [ ]:
sns.regplot(x='age',y='balance', data=df)
plt.show()

This plot looks messy, we will create a categorical column for 'age'.

In [ ]:
df['age_cate'] = pd.cut(df['age'], bins=[0, 35, 55, 70, float('Inf')], labels=['0-35', '35-55', '55-70', '70-95'])

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='age_cate', y='balance', hue='deposit', data=df, palette=palette)
plt.ylabel('balance($) log transform.')
plt.xlabel('Age')
plt.title('Balance VS Age by Deposit')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1.3, 0.6), title='Succes?', loc = 'lower right', fontsize=13, fancybox=True, shadow=True, frameon=True)
plt.show()

From this plot we can see that older customers tends to have a higher bank balance and that customers who accepted the deposit tended to have a higher balance compared to the customers who refused the deposit.

# Data Preparation for ML algorithms

In this section, we will perform some data preprocessing on the features. First, we will perform some transformations on skewed features.

# Categorical Features encoding

First, we encode the target variable.<br>
We note that the target column presents just two possible values: 'yes' or 'no'. Since these binary values appears in more columns, we encode all the 'yes' and 'no' in the whole dataframe with 0 and 1.

In [ ]:
df_clean = df_clean.replace(['yes','no'], [1,0])

Then, we can analysze the remaing categorical features.

In [ ]:
cat_feat = df_clean.select_dtypes(include="object").columns

In [ ]:
cat_feat

Depending on the categorical feature, we will choose if encoding it by one hot encoding or leabel encoding.

## Job

In [ ]:
df_clean['job'].value_counts()

There is no hierarchy, so this variable should be encoded by one hot encoding. Moreover, there are 63 samples with unknown jobs: we will label them as nan values and let XGBoost encode them.

In [ ]:
df_clean['job'] = df_clean['job'].replace('unknown', np.nan)

## Age cate

In [ ]:
df_clean['age_cate'].value_counts()

We can encode this feature by ordinal encoding

In [ ]:
df_clean['age_cate'] = df_clean['age_cate'].map({'0-35':0,'35-55':1,'55-70':2,'70-95':3})

## Marital status

In [ ]:
df_clean['marital'].value_counts()

We can encode this variable by ordinal encoding.

In [ ]:
df_clean['marital'] = df_clean['marital'].map({'single':0,'married':1,'divorced':2})

## Education

In [ ]:
df_clean['education'].value_counts()

We can encode this feature with ordinal encoding. Moreover, there are 417 samples with unknown education: they will be labeled as nan values.

In [ ]:
df_clean['education'] = df_clean['education'].map({'primary':0,'secondary':1,'tertiary':2, 'unknown':np.nan})

## Contact

In [ ]:
df_clean['contact'].value_counts()

There is no hierarchy among these features. However since there are just 2 features excluding 'unknown', we could impute these by 1 and 0. <br>
We will replace the rows with unknown contact with nan values.

In [ ]:
df_clean['contact'] = df_clean['contact'].map({'telephone':0,'cellular':1, 'unknown':np.nan})

## Month

In [ ]:
df_clean['month'].value_counts()

We can encode the month feature by ordinal label encoding.

In [ ]:
df_clean['month'] = df_clean['month'].map({'jan':0,'feb':1,'mar':2,'apr':3,'may':4,'jun':5,'jul':6,'aug':7,'sep':8,'oct':9,'nov':10,'dec':11})

## Poutcome

In [ ]:
df_clean['poutcome'].value_counts()

The great majority of customers have missing values for poutcome (over 7000) meaning that most of the customers have not been previously contacted. We will encode the feature by one hot encoding and drop the rows with 'poutcome='other' since it is not clear what it means.

In [ ]:
df_clean = df_clean[df_clean['poutcome']!='other']

In [ ]:
df_clean['poutcome'] = df_clean['poutcome'].replace('unknown',np.nan)

In [ ]:
df_clean.isna().sum()

We encode the remaining features with one hot encoding:

In [ ]:
encoded_df = pd.get_dummies(df_clean, drop_first=True)

In [ ]:
X = encoded_df.drop('deposit', axis=1)
y = encoded_df['deposit']

## Train - Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)

## Train - Validation split

Then we further split the train set into a new train and validation set to monitor the accuracy on the validation set and prevent overfitting.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.15, stratify=y_train, random_state=42)

# XGBoost

In [ ]:
def objective(trial, X_train=X_train, X_val=X_val, y_train=y_train, y_val=y_val):
    param = {
        'objective' : 'binary:logistic',
        'eval_metric' : 'auc',
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1,1),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-2,0.5),
        'n_estimators': trial.suggest_categorical('n_estimators', [1000,1500,2000,2500,3000,3500]),
        'max_depth': trial.suggest_int('max_depth', 3,10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
    }


    model = xgb.XGBClassifier(**param,use_label_encoder=False)    
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=False,)   
    preds = model.predict(X_val)
    score = accuracy_score(y_val, preds)

    return score

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")

hp = study.best_params

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
xgb_model = xgb.XGBClassifier(**hp)

In [ ]:
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=100, verbose=0)

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)

# Results Summary

In [ ]:
print(classification_report(y_test, y_pred_xgb,target_names = ['No','Yes']))

In [ ]:
plt.figure(figsize = (4, 5))

probs = xgb_model.predict_proba(X_test)
preds = probs[:,1]
fprxgb, tprxgb, _ = roc_curve(y_test, preds)
roc_aucxgb = auc(fprxgb, tprxgb)

plt.plot(fprxgb, tprxgb, 'b', label = 'AUC = %0.2f' % roc_aucxgb)
plt.plot([0, 1], [0, 1],'r--')
plt.title('ROC XGBoost ',fontsize=20)
plt.ylabel('True Positive Rate',fontsize=15)
plt.xlabel('False Positive Rate',fontsize=15)
plt.legend(loc = 'lower right', fontsize=15, fancybox=True, shadow=True, frameon=True,handlelength=0)
plt.show()

# Feature Importance

In [ ]:
xgb_model.feature_names = encoded_df.drop('deposit', axis = 1).columns
feat_df= pd.DataFrame({'feature': xgb_model.feature_names,'importance':xgb_model.feature_importances_})
sorted_df=feat_df.sort_values('importance', ascending=False)

In [ ]:
plt.figure(figsize=(6,3))
sns.barplot(x='importance', y='feature', data=sorted_df.head(10), palette='mako' )
plt.title('Feature Importance to predict success by XGBoost')
plt.xlabel('Feature Importance (%)')
plt.ylabel('')
plt.show()

- The most important features to determine if a new customer will accept the deposit is poutcome (>30%):<br>
    - **This means that the deposit satisfied the customers who accepted it previously, however, currently, the company is probably aiming at the 'wrong' customers** 
 <br>
- Moreover, accoring to XGBoost, other important features to predict the success of a deposit are:
    - Contact: by looking at the data it looks like the great majority of customers with unknown contact did not accept the deposit: it should be investigated why these contacts are missing.
    - Housing: customers without housing loans seems to accept the deposit
    

**Thanks for reading my notebook ! Let me know if you have questions or if you want me to check out your works !!**